<a href="https://colab.research.google.com/github/aredwing/Book-Recommender-System/blob/master/LFQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Long-Form Questioning Answering [LFQA] Application

## Dependencies

- Here, I'm going to install the dependencies that are helping me to build my pipeline. I'm using **Hay Stack** – it is an **open-source framework** for building search systems that work intelligently over large document collections.

- I wanted to use **ElasticSearch** for saving the documents after finishing the sentiment analysis but I have not gotten any answer from you!




In [ ]:
#@title (A) Check the GPU Hardware Specifications
!nvidia-smi

Thu May 26 12:45:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title (B) Connecting with Drive and create Main Directory
#@markdown Connect with GDrive if the dataset in GDrive
import os

if not os.path.exists("data"):
  os.mkdir("data")
connect_with_gdrive = True #@param {type: "boolean"}
if connect_with_gdrive:
  from google.colab import drive
  from IPython.display import clear_output
  print("Connecting with GDrive...")
  try:
    drive.mount('/content/drive')
    clear_output(wait=True)
    print('Successfully Connected with GDrive')
  except Exception as e:
    print(f"Failed to connect with Google Drive, Reason: {e}")
    

Successfully Connected with GDrive


In [ ]:
#@title ### (C) Installing Libraries
#@markdown <br>

#@markdown ####(1) Haystack framework

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,ocr]

!wget --no-check-certificate https://dl.xpdfreader.com/xpdf-tools-linux-4.04.tar.gz
!tar -xvf xpdf-tools-linux-4.04.tar.gz && sudo cp xpdf-tools-linux-4.04/bin64/pdftotext /usr/local/bin

# !pip install -q git+https://github.com/deepset-ai/haystack.git@a1040a17b2697bbaea2585dbcbf7cd175ce8cb75#egg=farm-haystack[colab]

# For FAISS DocumentStore
# !pip install -q git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss]

#@markdown ---
#@markdown ####(2) ElasticSearch DocumentStore
# In Colab / No Docker environments: Start Elasticsearch from source

! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30
#@markdown ---
# #@markdown ####(2) Install Dataset API
# # install datasets
# !pip install datasets

# # Make sure that we have a recent version of pyarrow in the session before we continue - otherwise reboot Colab to activate it
# import pyarrow
# if int(pyarrow.__version__.split('.')[1]) < 16 and int(pyarrow.__version__.split('.')[0]) == 0:
#     import os
#     os.kill(os.getpid(), 9)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 4.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-4g1orvpp/farm-haystack_92402322c32848b282aef1d8abdde33b
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-4g1orvpp/farm-haystack_92402322c32848b282aef1d8abdde33b
  Resolved https://github.com/deepset-ai/haystack.git to commit b2a2c10fae6944e8b38b7e72ef68a3f5f2586d6f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 71.

**All The Environment variables Initialized here in `utils.py`. Please, change it according to your AWS Environment.**

In [ ]:
#@title (D) Libraries
# %%writefile movie_reviewer/utils.py
import os
import sys
import re
import json
import warnings
from glob import glob
from collections import OrderedDict
from pprint import pprint
from typing import Text, List, Dict, Tuple, Optional, Union
warnings.filterwarnings("ignore")

import pandas as pd
from tqdm.auto import trange, tqdm

# Document Store
from haystack.utils import (launch_es,
                            print_answers,
                            print_documents,
                            convert_files_to_docs,
                            clean_wiki_text,
                            fetch_archive_from_http)

from haystack.telemetry import disable_telemetry
disable_telemetry()
# from haystack.document_stores import FAISSDocumentStore
from haystack.document_stores import ElasticsearchDocumentStore


# Nodes
## Preprocessing
from haystack.nodes import PreProcessor

## Retriever
from haystack.nodes import DensePassageRetriever

## Reader/Generator
from haystack.nodes import Seq2SeqGenerator

# Pipeline
from haystack.pipelines import (DocumentSearchPipeline,
                                GenerativeQAPipeline)

os.environ["HAYSTACK_TELEMETRY_ENABLED"] = "False"

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.
INFO - haystack.telemetry -  Telemetry has been disabled.


## 1. Data Ingestion

**Fetching paths required**

In [ ]:
# Datasets:
datasets = ["https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip",
            # Data is approx 5 GBs, it will need more time to be saved into the data store, we shall keep it for when the implementation is ready
            # "https://storage.googleapis.com/huggingface-nlp/datasets/bookcorpus/bookcorpus.tar.bz2", 
            # "https://thesbpermwhale.com/jaseweston/babi/movieqa.tar.gz" # Not Working
            ]

output_dirs = ["data/wikitext",
              #  "data/bookcorpus",
              #  "data/wikimovies"
               ]

In [ ]:
for dataset, dir in zip(datasets, output_dirs):
  fetch_archive_from_http(url=dataset, output_dir=dir)


INFO - haystack.utils.import_utils -  Fetching from https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip to `data/wikitext`


## 2. Data Preprocessing

### (A) Using Link

First, we need to convert `.row` datasets into `.txt` to be able to store them in the **Document Store**

In [ ]:
for data in os.listdir("data/wikitext/wikitext-2-raw"):
  with open(os.path.join("data/wikitext/wikitext-2-raw", data), 'r', encoding="utf-8") as rf:
    with open(os.path.join("data/wikitext/", "wikitext-2-raw.txt"), 'a', encoding="utf-8") as wf:
      wf.writelines("".join(rf.readlines()))


Remove the `.row` data now 

In [ ]:
!rm -rf /content/data/wikitext/wikitext-2-raw

**Convert datasets to documents**

In [ ]:
# Convert files to docs
docs = convert_files_to_docs(dir_path="data", #data
                             clean_func=clean_wiki_text,
                             split_paragraphs=True)

INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry
INFO - haystack.utils.preprocessing -  Converting data/wikitext/wikitext-2-raw.txt


## Adjust the file names

In [ ]:
#@markdown - Insert the directory of the files
FILES_PATH = "/content/drive/MyDrive/Work/Application_for_LFQA/Summary" #@param {"type": "string"}


def modify_names(filename: Text) -> Text:
  filename = re.sub(r"(\s\(\d\))", "", filename)
  filename = re.sub(r"(\s\-\s)(\w.+)(\.\w{3})", r"\3", filename)
  filename = re.sub(r"(\-)(\w.+)(\.\w{3})", r"\3", filename)
  filename = re.sub(r"(\-)|(\')", r"", filename)
  filename = filename.replace(" ", "_")
  return filename

if os.path.exists(FILES_PATH):
  for filename in os.listdir(FILES_PATH):
    os.rename(os.path.join(FILES_PATH, filename),
              os.path.join(FILES_PATH, modify_names(filename)))
else:
  print("the Directory you added is not found!")
    

### (B) Using Uploaded PDFs

In [ ]:
# Convert files to docs
docs = convert_files_to_docs(dir_path="/content/drive/MyDrive/Work/Application_for_LFQA/Summary") #data

INFO - haystack.utils.preprocessing -  Converting /content/drive/MyDrive/Work/Application_for_LFQA/Summary/1984.pdf
INFO - haystack.utils.preprocessing -  Converting /content/drive/MyDrive/Work/Application_for_LFQA/Summary/A_Beautiful_Mind.pdf
INFO - haystack.utils.preprocessing -  Converting /content/drive/MyDrive/Work/Application_for_LFQA/Summary/Dracula.pdf
INFO - haystack.utils.preprocessing -  Converting /content/drive/MyDrive/Work/Application_for_LFQA/Summary/Divergent.pdf
INFO - haystack.utils.preprocessing -  Converting /content/drive/MyDrive/Work/Application_for_LFQA/Summary/Dune.pdf
INFO - haystack.utils.preprocessing -  Converting /content/drive/MyDrive/Work/Application_for_LFQA/Summary/It.pdf
INFO - haystack.utils.preprocessing -  Converting /content/drive/MyDrive/Work/Application_for_LFQA/Summary/Misery.pdf
INFO - haystack.utils.preprocessing -  Converting /content/drive/MyDrive/Work/Application_for_LFQA/Summary/Seabiscuit.pdf
INFO - haystack.utils.preprocessing -  Convert

### **Preprocess the Documents**

In [ ]:
processor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=256,
    split_respect_sentence_boundary=True,
    split_overlap=0
)
processed_docs = processor.process(docs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


100%|██████████| 54/54 [00:02<00:00, 23.91docs/s]


## Creating & Launching DocumentStore 

In [ ]:
# Initialize DocumentStore and index documents
launch_es()
document_store = ElasticsearchDocumentStore(embedding_dim=128)
document_store.delete_documents()

WARNING - haystack.utils.doc_store -  Tried to start Elasticsearch through Docker but this failed. It is likely that there is already an existing Elasticsearch instance running. 


Let's add all the documents inside our ***DocumentStore***

In [ ]:
document_store.write_documents(processed_docs)

## Initialize Retriever and Generator

#### Retriever

We use a `DensePassageRetriever` and we invoke `update_embeddings` to index the embeddings of documents in the `FAISSDocumentStore`



In [ ]:
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="vblagoje/dpr-question_encoder-single-lfqa-wiki",
    passage_embedding_model="vblagoje/dpr-ctx_encoder-single-lfqa-wiki",
)

document_store.update_embeddings(retriever)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495 [00:00<?, ?B/s]

...

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find vblagoje/dpr-ctx_encoder-single-lfqa-wiki locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded vblagoje/dpr-ctx_encoder-single-lfqa-wiki
INFO - haystack.document_stores.elasticsearch -  Updating embeddings for all 4403 docs ...


Updating embeddings:   0%|          | 0/4403 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/4416 [00:00<?, ? Docs/s]

Before we blindly use the `DensePassageRetriever` let's empirically test it to make sure a simple search indeed finds the relevant documents.

In [ ]:
p_retrieval = DocumentSearchPipeline(retriever)
res = p_retrieval.run(query="What is this paper is sacking for?", params={"Retriever": {"top_k": 10}})
print_documents(res, max_text_len=512)


Query: What is this paper is sacking for?

{   'content': 'Paul tells Annie that editors hate getting manuscripts on this '
               'paper "almost as much as they hate handwritten manuscripts" '
               '(78). Annie sullenly agrees to get him better paper; Paul says '
               'it will be even better for binding, as she wants to do. As '
               'Annie leaves, she warns Paul that though she may look "slow '
               'and stupid," she can\'t be tricked (82). Suddenly, she charges '
               'at Paul in his wheelchair and punches him in the "bunched '
               'salt-dome" that had been his left knee. He screams in pain, '
               'and Annie tells him to go a...',
    'name': 'Misery.pdf'}

{   'content': 'He drops the match and the manuscript ignites. The papers '
               "blaze on Paul's board, turning the side of the Royal black. "
               "Flames begin to shoot up between the typewriter's keys. Annie "
               '

#### Generator

Similar to previous Tutorials we now initalize our generator.

Here we use a `Seq2SeqGenerator` with the *vblagoje/bart_lfqa* model (see: https://huggingface.co/vblagoje/bart_lfqa)



In [ ]:
generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1


Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

### Pipeline

With a Haystack `Pipeline` you can stick together your building blocks to a search pipeline.
Under the hood, `Pipelines` are Directed Acyclic Graphs (DAGs) that you can easily customize for your own use cases.
To speed things up, Haystack also comes with a few predefined Pipelines. One of them is the `GenerativeQAPipeline` that combines a retriever and a reader/generator to answer our questions.
You can learn more about `Pipelines` in the [docs](https://haystack.deepset.ai/docs/latest/pipelinesmd).

In [ ]:
pipe = GenerativeQAPipeline(generator, retriever)

### Testing the pipeline

#### (A) Streaming Generation

In [ ]:
predictions = pipe.run(
  # query="What is the benefit of life?",
  query="What movies you recommend me to watch?",
  params={"Retriever": {"top_k": 10}, "Generator": {"top_k": 1}})

In [ ]:
print_answers(predictions)


Query: What movies you recommend me to watch?
Answers:
[   <Answer {'answer': 'I\'m not sure if this counts as a "recommendation", but I\'d like to point out that there are a lot of movies that I\'ve never heard of that are very good. I\'m not saying that they\'re bad movies, but I\'m saying that I haven\'t heard of them.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_id': None, 'meta': {'doc_ids': ['71b5f77492ff0d848fc87338516fc8fe', '2fa29eb4ae39ed335bb0d747562a8b8a', '1ae11c9f52f081738e3f35b0f220eaa3', '5ae3a22d6533f63c43ba0448efb814ba', '450e73789c367674839b007d317e1953', '31de1bb05cc5851ca6028ed9252c80e8', 'e10900f760d62a4264ca4f7368b4b2c8', 'd3237fb3a4c65df07bed5b01a92c4881', 'a98edad7377a0a492bd50db6f14ea0e3', 'a0b0cd9002a0bf5025eeab2ba67cedd1'], 'doc_scores': [0.5470957256383229, 0.5426031969626242, 0.5423847860168193, 0.5413439844689028, 0.5398951858338268, 0.5395632697620654, 0.5393586522481174, 0.53

**Use this If you want Formatted Answer!**

In [ ]:
query="What is 'Fences'?"

predictions = pipe.run(
  # query="What is the benefit of life?",
  query=query,
  params={"Retriever": {"top_k": 10}, "Generator": {"top_k": 1}})

print("Question: ", query)
print()
print(" Answer ".center(25, "="))
pprint([answer.to_dict()['answer'] for answer in predictions['answers']][0])
print('*'*100)

Question:  What is 'Fences'?

========= Answer ========
('Troy Maxson is an African-American man who grew up in a segregated '
 'neighborhood in Pittsburgh, PA. He has a son, Cory, and a wife, Rose, who is '
 'white. Troy has a lot of problems in his life, and he feels like he has to '
 'do something about it. He decides to build a fence around his house to keep '
 "people out of his yard, but he doesn't know how to make it strong enough. He "
 "doesn't want his son to be afraid of him, so he keeps his son out of the "
 'house. He also has an affair with Rose, and when he finds out about it, he '
 "tells her that he's having an affair, and she tells him that she's pregnant, "
 "and that he can't have children with her.")
****************************************************************************************************


In [ ]:
print_documents(predictions)


Query: What is 'Fences'?

{   'content': '“Some people build fences to keep people out…and other people '
               'build fences to keep people in. Rose wants to hold onto you '
               'all. She loves you.” (Act II, Scene 1, Page 59)\n'
               'Bono has been hinting since the first scene of the play that '
               'he knows that Troy is having an affair. In this scene, Bono '
               'confronts Troy directly and elicits a confession. Bono '
               'prioritizes his own marriage and respects his wife. Troy has '
               'been reckless and risked his relationship with Rose because he '
               'feels trapped. Symbolically, this is why Troy is constantly '
               'building the fence but has not finished it. 16. “I ain’t '
               'ducking the responsibility of it. As long as it sets right in '
               'my heart…then I’m okay. ’Cause that’s all I listen to. It’ll '
               'tell me right from wrong every

#### (B) Batch Generation

In [ ]:
questions_list = ['What is "Fences"?.', 'Tell me about a movie called Fences.', 'Who are the characters of Fences?', "Explain to be the Plot Summary of the movie Fences."]

In [ ]:
from collections import defaultdict
QA = defaultdict(list)

for question in tqdm(questions_list, desc='Generating Answer(s)'):
  QA['Question'].append(question)
  prediction = pipe.run(query=question,
                                   params={"Retriever":
                                           {"top_k": 10},
                                           "Generator": {"top_k": 1}})
  QA['Answer'].append([answer.to_dict()['answer'] for answer in prediction['answers']][0])

Generating Answer(s):   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
for question, answer in list(zip(*list(dict(QA).values()))):
  # print("\n\n".join([question, answer]))
  print("Question: ", question)
  print()
  print(" Answer ".center(25, "="))
  pprint(answer)
  print('*'*100)
  

Question:  What is "Fences"?.

========= Answer ========
('"Fences" is a play by August Wilson. It\'s about a man named Troy Maxson, '
 'who grew up in a segregated neighborhood in Pittsburgh, PA. He grew up with '
 'his father, who was a racist, and his mother, who is a white woman. Troy was '
 'raised by his mother and father, and grew up as a victim of racism. He has a '
 'son, Cory, and a daughter, Rose. Troy and Rose have a child together, but '
 "Troy doesn't want his son to grow up to be a racist like his father. He "
 "doesn't like the idea of his son growing up with a racist father, so he "
 'builds a fence around his house to keep him out of the neighborhood. He also '
 'has an affair with his wife, Rose, who has a child with another man, Jim '
 "Bono. When Troy finds out that he's having an affair, he tells Rose, and she "
 "tells him that he can't marry her because she's pregnant. Troy")
***************************************************************************************